# Baseline model for batch monitoring example

In [ ]:
import requests
import pandas as pd
from tqdm import tqdm

In [ ]:
url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-03.parquet"
resp = requests.get(url, stream=True)
save_path = "./data/green_tripdata_2024-03.parquet"
with open(save_path, "wb") as handle:
    for data in tqdm(resp.iter_content(),
                    desc="green_tripdata_2024-03.parquet",
                    postfix=f"save to {save_path}",
                    total=int(resp.headers["Content-Length"])):
        handle.write(data)

green_tripdata_2024-03.parquet: 100%|██████████| 1372372/1372372 [00:06<00:00, 218141.67it/s, save to ./data/green_tripdata_2024-03.parquet]


In [30]:
mar_data = pd.read_parquet('data/green_tripdata_2024-03.parquet')
mar_data.shape

(57457, 20)

In [32]:
# create target
mar_data["duration_min"] = mar_data.lpep_dropoff_datetime - mar_data.lpep_pickup_datetime
mar_data.duration_min = mar_data.duration_min.apply(lambda td : float(td.total_seconds())/60)

# filter out outliers
mar_data = mar_data[(mar_data.duration_min >= 0) & (mar_data.duration_min <= 60)]
mar_data = mar_data[(mar_data.passenger_count > 0) & (mar_data.passenger_count <= 8)]
mar_data = mar_data[
    (mar_data['lpep_pickup_datetime'] >= '2024-03-01') &
    (mar_data['lpep_pickup_datetime'] < '2024-04-01')
]

mar_data['lpep_pickup_datetime'] = pd.to_datetime(mar_data['lpep_pickup_datetime'])
daily_medians = mar_data.groupby(mar_data['lpep_pickup_datetime'].dt.date)['fare_amount'].median()
max(daily_medians)

14.2